In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from models.yolov4 import CSPDarknet53_SPP_PAN
from models.common import *
from utility.loss_function import *
from utility.utilities import *
from utility.boxes import *
from utility.dataset import *
from utility.dataset_api import *
from utility.display import *

In [3]:
EPOCHS = 1

In [4]:
seed = 1
np.random.seed(seed)

torch.manual_seed(seed)
torch.set_deterministic(True)
# if use_cuda:
#     torch.cuda.manual_seed(seed)

In [5]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')
print("Using GPU: {}".format(use_cuda))

ds = CocoDatasetTrain()
# ds = CocoDatasetAPITrain()
# dl = load_dataloader(ds, batch_size=1)

modelTest = CSPDarknet53_SPP_PAN()
modelTest = modelTest.to(device=device)
# modelTest.load_state_dict(torch.load('model.pt', map_location=device))

optimiser = optim.Adam(modelTest.parameters())

Using GPU: False


In [15]:
from scratch import Yolo_loss

dl = DataLoader(ds, batch_size=3, num_workers=0, worker_init_fn=np.random.seed(seed), collate_fn=ds.collate_fn, \
                sampler=torch.utils.data.RandomSampler(data_source=ds, generator=torch.Generator().manual_seed(seed)))

yolo_loss = Yolo_loss()
# print(yolo_loss.ref_anchors[2][-3:])

for epoch in range(EPOCHS):
    for i, batch in enumerate(dl):
        images, targets = batch
        # images = images.to(device=device)
        # targets = targets.to(device=device)

        # print("Targets:")
        # print(targets)
        
        preds = modelTest(images)[-1]

        # # predsGithub = torch.clone(preds)
        
        preds = split_output(preds, device)
        
        preds[0][:,:,:,:,0:2] = ACTIVATIONS['sigmoid'](preds[0][:,:,:,:,0:2])
        preds[1] = ACTIVATIONS['sigmoid'](preds[1])
        preds[2] = ACTIVATIONS['sigmoid'](preds[2])
        
        pred_boxes = prediction_to_boxes(preds[0], 's_scale', device)

        # preds_github_boxes = yolo_loss(predsGithub).permute(0,2,3,1,4)
        
        masks_and_target = build_target(pred_boxes, preds[2], targets, 's_scale', device)

        # object_mask, no_object_mask, class_mask, ious_pred_target, \
        #     target_x, target_y, target_w, target_h, target_obj, target_class_1hot = masks_and_target
        
        # loss = loss_function2(pred_boxes, preds, masks_and_target, device)
        # loss = loss_function(preds, masks_and_target, device)
        # print(loss)      

        # show_boxes(images.squeeze(0), targets)
        
        # optimiser.zero_grad()
        # loss.backward()
        # optimiser.step()

        # print('Iteration {} done, loss: {}'.format(i, loss.item()))

        break


NameError: name 'x2' is not defined

In [12]:
# torch.save(modelTest.state_dict(), 'model.pt')